In [17]:
import cv2
import numpy as np
import time
from scipy import ndimage as ndi
from skimage.morphology import watershed
from skimage.feature import peak_local_max
import pandas as pd
from scipy.ndimage import label
import sys
import collections
from scipy.spatial import distance as dist
from collections import OrderedDict
from scipy.ndimage import gaussian_filter

In [18]:
def make_boxes(color_img,image,labels):       
    conts=[]    
    contours=[]
    for label in np.unique(labels):
        # if the label is zero, we are examining the 'background'
        # so simply ignore it
        if label == 0:
            continue
        # otherwise, allocate memory for the label region and draw
        # it on the mask
        mask = np.zeros(gray.shape, dtype="uint8")
        mask[labels == label] = 255
        # detect contours in the mask and grab the largest one
        cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
            cv2.CHAIN_APPROX_SIMPLE)[-2]
        c = max(cnts, key=cv2.contourArea)
        
#         rect = cv2.minAreaRect(c)
#         box = cv2.boxPoints(rect)
#         box = np.int0(box)  
#         if cv2.contourArea(c) > 50: 
#             #cv2.drawContours(image,c,-1,(0,255,0))
#             cv2.drawContours(image,[box],-1,(128,128,128),thickness = 1)
        hull = cv2.convexHull(c)
        #cv2.drawContours(image,[hull],-1,(0,0,255),thickness = 1)
        contours.append(hull)
    for i in range(len(contours)):
        c = contours[i]

        area = cv2.contourArea(c)

        # Iterate all contours from i+1 to end of list
        for j in range(i+1, len(contours)):
            c2 = contours[j]

            area2 = cv2.contourArea(c2)

            area_sum = area + area2

            # Merge contours together
            tmp = np.vstack((c, c2))
            merged_c = cv2.convexHull(tmp)

            merged_area = cv2.contourArea(merged_c)

            # Replace contours c and c2 by the convex hull of merged c and c2, if total area is increased by no more then 10%
            if merged_area < area_sum*1.1:
                # Replace contour with merged one.
                contours[i] = merged_c
                contours[j] = merged_c
                c = merged_c
                area = merged_area
    ################################################################################


    # Draw new contours in red color
    for c in contours:
        #Ignore small contours
        if cv2.contourArea(c) > 20:
            cv2.drawContours(color_img, [c], -1, (255,255,0), 2, 1)
            cv2.drawContours(image, [c], -1, (128,128,128), 2, 1)
    return image,color_img
def remove_background(image,back_img):
    h,w = image.shape
    output = np.zeros((h,w),dtype = "uint8")
    output = np.subtract(image,back_img)
    return output
def apply_watershed(img):
    img_array = img.copy()
    distance = ndi.distance_transform_edt(img_array)
    markers = ndi.label(peak_local_max(distance, min_distance = 10,footprint = np.ones((11, 11)), indices = False, labels = img_array))[0]
    ws_labels = watershed(-distance, markers, mask = img_array)
    print("[INFO] {} unique segments found".format(len(np.unique(ws_labels)) - 1))
    return ws_labels,markers
def segmentation_try_1(frame,gray):
        image = frame.copy()
        #applying min_max filtering
        #frame = run_algo(gray, filter_size,M)
        kernel_erosion = np.ones((3,3),np.uint8)
        erosion = cv2.erode(gray,kernel_erosion,iterations = 1)
        kernel_dilation = np.ones((5,5), np.uint8)
        img_dilation = cv2.dilate(erosion, kernel_dilation, iterations=1)
        th = cv2.adaptiveThreshold(img_dilation,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
        frame = remove_background(th,img_dilation)
        blur = cv2.GaussianBlur(frame,(5,5),0)
        ret,th = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        frame = cv2.bitwise_not(th)
        #frame = frst(frame,2,2,0.25,0.25)
        label,markers = apply_watershed(frame)
       # print("labels",label)
       # print("markers",markers)
        #pd.DataFrame(markers).to_csv("markers.csv")
        #pd.DataFrame(label).to_csv("labels.csv")
        frame_boxes,color_img = make_boxes(image,frame,label)
        return color_img,frame_boxes,markers

In [19]:
def segmentation_try_2(frame,gray):
        ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        # noise removal
        kernel = np.ones((3,3),np.uint8)
        opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)
        # sure background area
        sure_bg = cv2.dilate(opening,kernel,iterations=3)
        # Finding sure foreground area
        dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
        ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)
        # Finding unknown region
        sure_fg = np.uint8(sure_fg)
        unknown = cv2.subtract(sure_bg,sure_fg)
        # Marker labelling
        ret, markers = cv2.connectedComponents(sure_fg)
        # Add one to all labels so that sure background is not 0, but 1
        markers = markers+1
        # Now, mark the region of unknown with zero
        markers[unknown==255] = 0
        markers = cv2.watershed(frame,markers)
        #frame_boxes = make_boxes(frame,markers)
        frame[markers == -1] = [255,0,0]
        df = pd.DataFrame(markers)
        df.to_csv("markers.csv")
        return frame

In [20]:
def segmentation_try_3(frame):
        image = frame.copy()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
                
        filtersize = (5,5)
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT,filtersize)
        frame = cv2.erode(frame, kernel, iterations=1)
        
        filtersize = (3,3)
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT,filtersize)


        th = cv2.morphologyEx(frame,cv2.MORPH_TOPHAT,kernel)

        
        thresh = cv2.adaptiveThreshold(th,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
        
        
        gf = gaussian_filter(th,3)
        
        sub = cv2.subtract(gf,3)
        frame = cv2.dilate(sub, kernel, iterations=1)

        frame_thresh = np.where(frame < 1,sub,255)
        

        return frame_thresh

In [21]:
def find_centroid(markers):

    markers = pd.DataFrame(markers)
    markers.drop(markers.columns[[0]],axis=1,inplace=True)
    all_point_dict = collections.defaultdict(list)
    centroid_dict = collections.defaultdict(list)

    for i,row in markers.iterrows():
        for j, val in enumerate(row):
            all_point_dict[val].append([i,j])
        #break

            #a[markers.iloc[i,j]].append([i,j])

    for key in all_point_dict:
        if key == 0:
            continue
        else:
            if len(all_point_dict[key])%2 == 0:
                centroid_dict[key] = all_point_dict[key][int(len(all_point_dict[key])/2)-1]
            else:
                centroid_dict[key] = all_point_dict[key][int((len(all_point_dict[key])+1)/2)-1]
                
    centrelist = list()
    for key,value in centroid_dict.items():
        centrelist.append(value)
                
    return centrelist

In [22]:
class CentroidTracker():
    def __init__(self, maxDisappeared=10):
        # initialize the next unique object ID along with two ordered
        # dictionaries used to keep track of mapping a given object
        # ID to its centroid and number of consecutive frames it has
        # been marked as "disappeared", respectively
        self.nextObjectID = 1
        self.objects = OrderedDict()
        self.disappeared = OrderedDict()
        
        self.colors = OrderedDict()
        self.coords = OrderedDict()
        self.info = OrderedDict()

        # store the number of maximum consecutive frames a given
        # object is allowed to be marked as "disappeared" until we
        # need to deregister the object from tracking
        self.maxDisappeared = maxDisappeared

    def register(self, centroid):
        
        # when registering an object we use the next available object
        # ID to store the centroid
        self.objects[self.nextObjectID] = centroid
        self.disappeared[self.nextObjectID] = 0
        
        self.colors[self.nextObjectID] = tuple(np.random.choice(range(256), size=3))
        
        coord_list = list()
        coord_list.append(centroid)
        self.coords[self.nextObjectID] = coord_list
        
        cell_info = {'speed': 0.0, 'tDist': 0.0, 'nDist': 0.0, 'ratio': 0.0}
        self.info[self.nextObjectID] = cell_info
        
        self.nextObjectID += 1
        
    def deregister(self, objectID):
        # to deregister an object ID we delete the object ID from
        # both of our respective dictionaries
        del self.objects[objectID]
        del self.disappeared[objectID]
        del self.colors[objectID]
        del self.coords[objectID]
        del self.info[objectID]

    def update(self, inputCentroids):
        # check to see if the list of input bounding box rectangles
        # is empty
        if len(inputCentroids) == 0:
            # loop over any existing tracked objects and mark them
            # as disappeared
            for objectID in list(self.disappeared.keys()):
                self.disappeared[objectID] += 1

                # if we have reached a maximum number of consecutive
                # frames where a given object has been marked as
                # missing, deregister it
                if self.disappeared[objectID] > self.maxDisappeared:
                    self.deregister(objectID)

            # return early as there are no centroids or tracking info
            # to update
            return self.objects

        # initialize an array of input centroids for the current frame
        #inputCentroids = np.zeros((len(rects), 2), dtype="int")

        # loop over the bounding box rectangles
#         for (i, (startX, startY, endX, endY)) in enumerate(rects):
#             # use the bounding box coordinates to derive the centroid
#             cX = int((startX + endX) / 2.0)
#             cY = int((startY + endY) / 2.0)
#             inputCentroids[i] = (cX, cY)

        # if we are currently not tracking any objects take the input
        # centroids and register each of them
        if len(self.objects) == 0:
            for i in range(0, len(inputCentroids)):
                self.register(inputCentroids[i])
        # otherwise, are are currently tracking objects so we need to
        # try to match the input centroids to existing object
        # centroids
        else:
            # grab the set of object IDs and corresponding centroids
            objectIDs = list(self.objects.keys())
            objectCentroids = list(self.objects.values())

            # compute the distance between each pair of object
            # centroids and input centroids, respectively -- our
            # goal will be to match an input centroid to an existing
            # object centroid
            D = dist.cdist(np.array(objectCentroids), inputCentroids)

            # in order to perform this matching we must (1) find the
            # smallest value in each row and then (2) sort the row
            # indexes based on their minimum values so that the row
            # with the smallest value as at the *front* of the index
            # list
            rows = D.min(axis=1).argsort()

            # next, we perform a similar process on the columns by
            # finding the smallest value in each column and then
            # sorting using the previously computed row index list
            cols = D.argmin(axis=1)[rows]

            # in order to determine if we need to update, register,
            # or deregister an object we need to keep track of which
            # of the rows and column indexes we have already examined
            usedRows = set()
            usedCols = set()

            # loop over the combination of the (row, column) index
            # tuples
            for (row, col) in zip(rows, cols):
                # if we have already examined either the row or
                # column value before, ignore it
                # val
                if row in usedRows or col in usedCols:
                    continue

                # otherwise, grab the object ID for the current row,
                # set its new centroid, and reset the disappeared
                # counter
                objectID = objectIDs[row]
                
     ###############Task 3           
                # calculate distance between previous and current
                # coords and update cell info
                path_dist = cv2.norm(np.subtract(self.objects[objectID], inputCentroids[col]))
                norm_dist = cv2.norm(np.subtract(self.coords[objectID][0], inputCentroids[col]))
                self.info[objectID]['speed'] = path_dist
                self.info[objectID]['tDist'] += path_dist
                self.info[objectID]['nDist'] = norm_dist
                if (norm_dist != 0):
                    self.info[objectID]['ratio'] = self.info[objectID]['tDist']/norm_dist
    ###############         
                
                self.objects[objectID] = inputCentroids[col]
                self.coords[objectID].append(inputCentroids[col])
                self.disappeared[objectID] = 0

                # indicate that we have examined each of the row and
                # column indexes, respectively
                usedRows.add(row)
                usedCols.add(col)

            # compute both the row and column index we have NOT yet
            # examined
            unusedRows = set(range(0, D.shape[0])).difference(usedRows)
            unusedCols = set(range(0, D.shape[1])).difference(usedCols)

            # in the event that the number of object centroids is
            # equal or greater than the number of input centroids
            # we need to check and see if some of these objects have
            # potentially disappeared
            if D.shape[0] >= D.shape[1]:
                # loop over the unused row indexes
                for row in unusedRows:
                    # grab the object ID for the corresponding row
                    # index and increment the disappeared counter
                    objectID = objectIDs[row]
                    self.disappeared[objectID] += 1

                    # check to see if the number of consecutive
                    # frames the object has been marked "disappeared"
                    # for warrants deregistering the object
                    if self.disappeared[objectID] > self.maxDisappeared:
                        self.deregister(objectID)

            # otherwise, if the number of input centroids is greater
            # than the number of existing object centroids we need to
            # register each new input centroid as a trackable object
            else:
                for col in unusedCols:
                    self.register(inputCentroids[col])
        
        #######task 3
        info = "Cell ID 38, speed: {} tDist: {} nDist: {} ratio: {}".format(self.info[38]['speed'], self.info[38]['tDist'],self.info[38]['nDist'],self.info[38]['ratio'])           
        print(info)
        ######
        # return the set of trackable objects
        return self.objects
    
    def get_color(self, objectID):
        return self.colors[objectID]
    def get_coord(self, objectID):
        return self.coords[objectID]

In [23]:
def draw_line(image, coord1, coord2, color):
    cv2.line(image, (int(coord1[1]), int(coord1[0])), (int(coord2[1]), int(coord2[0])) , color)

In [24]:
cap = cv2.VideoCapture("C:/Users/Ujjwal/Desktop/Subjects/Term5/Assignment 2/images/PhC-C2DL-PSC/Sequence 4/t%03d.tif")
filter_size = 3
ct = CentroidTracker()
while(1):
    ret, frame = cap.read()
    if (ret) :
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
#####################Section 1#########################################################      

#         segmentation_try_1_img,Black,markers = segmentation_try_1(frame,gray)
#         inputCentroids = find_centroid(markers)

#####################Section 2#######################################################
        
        #frame = segmentation_try_2(frame,gray)
        #_, contours, _ = cv2.findContours(frame,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        
####################Sectior 3########################################################

        frame_thresh = segmentation_try_3(frame)
        _, contours, _ = cv2.findContours(frame_thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        inputCentroids = []
        for contour in contours:
            (x,y), radius = cv2.minEnclosingCircle(contour)
            center = (x, y)
            inputCentroids.append(center)

        #...
        #...
        #...
##################Tracking from centroids########################################################        
        
        objects = ct.update(inputCentroids)
        for (objectID, centroid) in objects.items():
        # draw both the ID of the object and the centroid of the
        # object on the output frame
            color = (int(ct.get_color(objectID)[1]), int(ct.get_color(objectID)[0]), int(ct.get_color(objectID)[2]))

            text = "ID {}".format(objectID)
            #print(int(centroid[1]),int(centroid[0]))
            cv2.putText(frame, text, (int(centroid[0]), int(centroid[1])),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            
            cv2.circle(frame, (int(centroid[0]), int(centroid[1])), 4, color, 1)
            
            for ind, coord in enumerate(ct.get_coord(objectID)):
                if ind < len(ct.get_coord(objectID)) - 1:
                    draw_line(frame, coord, ct.get_coord(objectID)[ind + 1], color)

        # show the output frame
        cv2.imshow("Frame", frame)
        
        time.sleep(1)
        
    else:
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    # 'q' to quit playback
    if (cv2.waitKey(1) & 0xFF == ord('q')):
        break
        
cap.release()
cv2.destroyAllWindows()

Cell ID 38, speed: 0.0 tDist: 0.0 nDist: 0.0 ratio: 0.0
Cell ID 38, speed: 0.5 tDist: 0.5 nDist: 0.5 ratio: 1.0
Cell ID 38, speed: 0.5 tDist: 1.0 nDist: 0.7071067811865476 ratio: 1.414213562373095
Cell ID 38, speed: 0.979743178491244 tDist: 1.979743178491244 nDist: 0.9376773095344425 ratio: 2.111326741471635
